# Trabajo de Fin de Grado
### Grado en Ingeniería Informática

## Estudio del problema de la generalización de modelos de aprendizaje profundo entrenados para el diagnóstico del glaucoma.

### Eduardo González Gutiérrez

---

## Importaciones

In [1]:
import os
import math
import scipy
import shutil
import random
import tensorflow as tf 
from pathlib import Path
from tensorflow import keras 
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K  
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Dropout 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50 
from tensorflow.keras.callbacks import ModelCheckpoint 
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Mostramos la versión de TensorFlow que se va a utilizar
print("Versión de TensorFlow:", tf.__version__) 

# Mostramos la GPU que se va a utilizar
print("GPUs disponibles:", tf.config.list_physical_devices('GPU')) 

## Rutas Datasets

In [ ]:
base_dir = os.getcwd()

# Definimos las rutas del directorio
fold_dir = os.path.join(base_dir, 'Dataset', 'Entrenamientos', 'Imagenes-Segmentadas', 'validacion_cruzada', 'fold_5')

# Dentro del directorio fold_dir, accedemos a los subdirectorios train y val
train_dir = os.path.join(fold_dir, 'train')
validation_dir = os.path.join(fold_dir, 'val')

# Definimos la ruta del directorio de test
test_dir = os.path.join(base_dir, 'Dataset', 'Evaluacion', 'REFUGE', 'Mascaras')


# Diccionario con las rutas
dirs = {'Train': train_dir, 'Validation': validation_dir, 'Test': test_dir}

# Definimos las clases de imágenes en las cuales se clasifcan las imágenes en los directorios
clases = ['Normales', 'Glaucomas']

# Imprimimos las rutas de los directorios y el número de imágenes en cada clase
for nombre_dir, ruta_dir in dirs.items():
    print(f"\n--- {nombre_dir} ---")
    print("Ruta:", ruta_dir)
    for clase in clases:
        ruta_clase = os.path.join(ruta_dir, clase)
        if os.path.exists(ruta_clase):
            num_imagenes = len([
                f for f in os.listdir(ruta_clase)
                if os.path.isfile(os.path.join(ruta_clase, f))
            ])
            print(f"{clase}: {num_imagenes} imágenes")
        else:
            print(f"No se encontró la carpeta {ruta_clase}")

## Funciones de preprocesado de imágenes 

In [ ]:
# Función de preprocesado para redes ResNet50
def preprocess_input(x):
  # Obtenemos el formato de los datos de imagen
  data_format = K.image_data_format()
  assert data_format in {'channels_last', 'channels_first'}

  # Preprocesamos la imagen según el formato de datos
  # En el formato 'channels_first', la imagen tiene la forma (canales, alto, ancho)
   # En el formato 'channels_last', la imagen tiene la forma (alto, ancho, canales)
  if data_format == 'channels_first':
    # Convertimos de 'RGB' a 'BGR' invirtiendo el orden de los canales
    x = x[::-1, :, :]
    
    # Centramos los valores de píxel restando la media por canal.
    x[0, :, :] -= 103.939
    x[1, :, :] -= 116.779
    x[2, :, :] -= 123.68
  else:
    # Convertimos de 'RGB' a 'BGR' invirtiendo el eje de los canales
    x = x[:, :, ::-1]

    # Centramos los valores de píxel restando la media por canal.
    x[:, :, 0] -= 103.939
    x[:, :, 1] -= 116.779
    x[:, :, 2] -= 123.68
  return x

## Función de Construcción del Modelo

In [ ]:
# Función para construir el modelo ResNet50 con las capas de salida personalizadas adaptadas a nuestro problema de clasificación binaria (glaucoma vs no glaucoma)
def build_model(input_shape_size, weights_source, trainable_condition):
    
    # Cargamos el modelo preentrenado ResNet50 sin la capa de salida (include_top=False) y especificamos el tamaño de entrada (input_shape_size).
    pre_trained_model = ResNet50(input_shape=input_shape_size, include_top=False, weights=weights_source)

    # Definimos si las capas del modelo preentrenado deben ser entrenables o no.
    pre_trained_model.trainable = trainable_condition

    # Añadimos una capa de normalización por lotes (Batch Normalization) para mejorar la estabilidad del entrenamiento
    x = GlobalAveragePooling2D()(pre_trained_model.output)

    # Añadimos un Dropout para reducir el sobreajuste
    x = Dropout(0.3)(x)

    # Añadimos una nueva capa final que clasifique en 2 clases (glaucoma y no glaucoma)
    outputs = Dense(2, activation='softmax')(x)

    # Creamos el modelo final combinando la entrada del modelo preentrenado y la salida personalizada
    model = Model(inputs=pre_trained_model.input, outputs=outputs)

    return model

## Data Augmentation

In [ ]:
# Generadores de imágenes con redimensionamiento y normalización para ResNet50
image_size = (224, 224)

# Creamos el objeto ImageDataGenerator para el Data Augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, # Llamamos a función de preprocesado
    rotation_range=30, # Rotación entre -20º y 20º
    horizontal_flip=True, # Flip horizontal
    brightness_range=[0.8, 1.2], # Variación de brillo
)

# Generador para validación y test, solo con preprocesamiento, sin aumentos de datos
validation_test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Creamos el generador que cargará las imágenes de entrenamiento, redimensiona las imágenes, aplica aumentos, las agrupa en batches de 32 y define las clases objetivo
train_generator = train_datagen.flow_from_directory(train_dir, target_size=image_size, batch_size=32, classes=['Normales', 'Glaucomas'])

# Generador para el conjunto de validación, sin aumentos, solo preprocesamiento
validation_generator = validation_test_datagen.flow_from_directory(validation_dir, target_size=image_size, batch_size=32, classes=['Normales', 'Glaucomas'])

# Generador para el conjunto de test, sin barajar las imágenes (shuffle=False) para evaluación ordenada
test_generator = validation_test_datagen.flow_from_directory(test_dir, target_size=image_size, batch_size=32, classes=['Normales', 'Glaucomas'], shuffle=False)

## Fine Tuning

In [ ]:
# Definimos la forma de entrada que tendrá la red: imágenes RGB de 224x224 píxeles
input_shape = (224, 224, 3)

# Indicamos que usaremos pesos preentrenados en ImageNet para la inicialización del modelo
weights = 'imagenet'

# Indicamos que las capas preentrenadas no serán entrenables.
trainable_condition = False

# Llamamos a la función que define el modelo
model = build_model(input_shape, weights, trainable_condition)

# Se muestra un resumen de la red
model.summary()

In [ ]:
# Configuramos el entrenamiento de la red con los siguientes parámetros:
#   - loss='categorical_crossentropy': Función de pérdida para clasificación binaria (glaucoma vs no glaucoma)
#   - optimizer=Adam: Optimizador que ajusta los pesos
#   - learning_rate=1e-4: Tasa de aprendizaje
#   - metrics: Accuracy: Porcentaje de aciertos

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=1e-4),   
              metrics=['accuracy'])

In [ ]:
model_name='ResNet50-FineTuning-Fold5-600-epochs-Originales'

# Definimos un callback para guardar automáticamente el mejor modelo durante el entrenamiento
checkpoint = ModelCheckpoint(
        model_name,  # Nombre del archivo donde se guardará el mejor modelo
        monitor="val_accuracy",  # Monitoreamos la precisión en validación
        save_best_only=True,  # Guarda solo si es el mejor hasta el momento
        mode="max",  # Queremos la mayor precisión posible
        verbose=1,  # Muestra mensajes cuando guarda un nuevo mejor modelo
)

# Entrenamos el modelo usando el generador de entrenamiento y validación
history_fine_tuning = model.fit(
    train_generator, # Datos de entrenamiento con aumentos y preprocesamiento
    epochs=600, # Número total de épocas para entrenar
    batch_size=32, # Tamaño de lote para cada iteración
    callbacks=[checkpoint], # Callback para guardar el mejor modelo durante el entrenamiento
    validation_data=validation_generator, # Datos para validación al final de cada época
)

In [ ]:
# Cargamos el mejor modelo guardado durante el entrenamiento
model.load_weights(model_name) 

In [ ]:
# Evaluación del modelo
# Primero evaluamos el último modelo por si acaso sea el mejor
# Después evaluamos el mejor modelo guardado

test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")
print(f"Test loss: {test_loss}")

# Obtener la cantidad de épocas realmente entrenadas
epochs_trained = len(history_fine_tuning.history['loss'])
print(f"El entrenamiento se detuvo en la época: {epochs_trained}")

In [ ]:
# Gráficos de entrenamiento
acc = history_fine_tuning.history['accuracy']
val_acc = history_fine_tuning.history['val_accuracy']
loss = history_fine_tuning.history['loss']
val_loss = history_fine_tuning.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.savefig('Resultados/ResNet50/Imagenes-Originales/Fine-Tuning/ResNet50-FineTuning-Fold5-600-epochs-Graph.png')  # Guardar la figura
plt.show()

In [ ]:
# Guarda solo los pesos del modelo
model.save_weights('Resultados/ResNet50/Imagenes-Originales/Fine-Tuning/ResNet50-FineTuning-Fold5 -600-epochs.h5')  

## Deep Tuning

In [ ]:
# Definimos la forma de entrada que tendrá la red: imágenes RGB de 224x224 píxeles
input_shape = (224, 224, 3)

# Indicamos que usaremos pesos preentrenados en ImageNet para la inicialización del modelo
weights = 'imagenet'

# Indicamos que las capas preentrenadas sí serán entrenables.
trainable_condition = True

# Llamamos a la función que define el modelo
model = build_model(input_shape, weights, trainable_condition)

# Cargamos los pesos del modelo previamente entrenado con fine-tuning
model.load_weights('Resultados/ResNet50/Imagenes-Segmentadas/Fine-Tuning/ResNet50-FineTuning-Fold2-600-epochs.h5') 

# Se muestra un resumen de la red
model.summary()

In [ ]:
# Configuramos el entrenamiento de la red con los siguientes parámetros:
#   - loss='categorical_crossentropy': Función de pérdida para clasificación binaria (glaucoma vs no glaucoma)
#   - optimizer=Adam: Optimizador que ajusta los pesos
#   - learning_rate=1e-5: Tasa de aprendizaje
#   - metrics: Accuracy: Porcentaje de aciertos

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=1e-5),   
              metrics=['accuracy'])

In [ ]:
model_name='ResNet50-DeepTuning-Fold5-PeorFineTuning-600-epochs'

# Definimos un callback para guardar automáticamente el mejor modelo durante el entrenamiento
checkpoint = ModelCheckpoint(
        model_name,  # Nombre del archivo donde se guardará el mejor modelo
        monitor="val_accuracy",  # Monitoreamos la precisión en validación
        save_best_only=True,  # Guarda solo si es el mejor hasta el momento
        mode="max",  # Queremos la mayor precisión posible
        verbose=1,  # Muestra mensajes cuando guarda un nuevo mejor modelo
)

# Entrenamos el modelo usando el generador de entrenamiento y validación
history_fine_tuning = model.fit(
    train_generator, # Datos de entrenamiento con aumentos y preprocesamiento
    epochs=600, # Número total de épocas para entrenar
    batch_size=32, # Tamaño de lote para cada iteración
    callbacks=[checkpoint], # Callback para guardar el mejor modelo durante el entrenamiento
    validation_data=validation_generator, # Datos para validación al final de cada época
)

In [ ]:
# Cargamos el mejor modelo guardado durante el entrenamiento
model.load_weights(model_name)

In [ ]:
# Evaluación del modelo
# Primero evaluamos el último modelo por si acaso sea el mejor
# Después evaluamos el mejor modelo guardado

test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")
print(f"Test loss: {test_loss}")

# Obtener la cantidad de épocas realmente entrenadas
epochs_trained = len(history_fine_tuning.history['loss'])
print(f"El entrenamiento se detuvo en la época: {epochs_trained}")

In [ ]:
# Gráficos de entrenamiento
acc = history_fine_tuning.history['accuracy']
val_acc = history_fine_tuning.history['val_accuracy']
loss = history_fine_tuning.history['loss']
val_loss = history_fine_tuning.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.savefig('Resultados/ResNet50/Imagenes-Segmentadas/Deep-Tuning/Peor-Fine-Tuning/ResNet50-DeepTuning-Fold5-600-epochs-Graph.png')  # Guardar la figura
plt.show()

In [ ]:
# Guarda solo los pesos del modelo
model.save_weights('Resultados/ResNet50/Imagenes-Segmentadas/Deep-Tuning/Peor-Fine-Tuning/ResNet50-DeepTuning-Fold5-600-epochs.h5')

## Evaluación del Modelo

In [ ]:
# Definimos la forma de entrada que tendrá la red: imágenes RGB de 224x224 píxeles
input_shape = (224, 224, 3)

# Indicamos que usaremos pesos preentrenados en ImageNet para la inicialización del modelo
weights = 'imagenet'

# Indicamos si las capas preentrenadas serán entrenadas o no. 
trainable_condition = False

# Llamamos a la función que define el modelo
model = build_model(input_shape, weights, trainable_condition)

# Cargamos los pesos del modelo previamente entrenado y que ha dado mejores resultados entre los deep tunings y fine tunings
model.load_weights('Resultados/ResNet50/Imagenes-Segmentadas/Fine-Tuning/ResNet50-FineTuning-Fold1-600-epochs.h5')

# Se muestra un resumen de la red
model.summary()

# Configuramos el entrenamiento de la red con los siguientes parámetros:
#   - loss='categorical_crossentropy': Función de pérdida para clasificación binaria (glaucoma vs no glaucoma)
#   - optimizer=Adam: Optimizador que ajusta los pesos
#   - learning_rate=1e-5: Tasa de aprendizaje
#   - metrics: Accuracy: Porcentaje de aciertos
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=1e-5),   
              metrics=['accuracy'])

In [ ]:
# Evaluación del modelo
print("Total test images:", test_generator.samples)
test_loss, test_acc = model.evaluate(test_generator, verbose=1)  # verbose=1 muestra progreso
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test loss: {test_loss:.4f}")